### 시장 시뮬레이터
* 일별 OHLC 데이터 제공
* 전략별 매매결과 및 분석 데이터 제공 

In [2]:
import os, sys, pickle
sys.path.append('../../')
sys.path.append('../../scripts')
from collections import OrderedDict, defaultdict
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models import OHLC, Products, Product
from tools import ohlc_chart, view

In [3]:
""" DB 로드"""
products = pickle.load(open('../../data/products.pkl', 'rb')) #종목정보
#db = tb.open_file('../data/rawminute.db', mode='r')
#filters = tb.Filters(complib='blosc', complevel=9)
#db = tb.open_file('../data/rawminute.db', mode='a', filters=filters) #분데이터 db 로드

In [5]:
BUY = 1
SELL = -1

In [6]:
class Invoice:
    """
    1계약의 정보를 담고 있는 거래명세서
    명세서번호, 사용자이름, 상품명, 포지션, 매입가격, 매입날짜, 당일기준손익을 나타내는 클래스
    """
    
    def __init__(self,pk, name, product, position, entryprice, entrydate):
        self.id = pk #명세서 번호
        self.name = name #소유자 (system)
        self.product = product #상품
        self.position = position #포지션
        self.entryprice = entryprice #매입가격
        self.entrydate = entrydate #매입날짜
        self.status = True #매매상태
        
    def margin(self, curprice):
        """ 
        매입가격과 현재가격(curprice) 의 차액
        """
        price_diff = round(self.entryprice - curprice, self.product.decimal_len)
        return self.position * price_diff * self.product.tickprice / self.product.tickunit
    
    def exit(self, price, date):
        """
        invoice 매매 종료
        """
        self.exitprice = price #청산가
        self.exitdate = date #청산일
        self.profit = self.margin(price) #손익
        self.status = False #거래진행상태

In [7]:
class Exchange:
    """
    거래소 클래스 
    과거 선물데이터를 제공하고, 매수매도 주문접수를 받는다.
    모든 거래 기록(invoice)을 저장한다.
    """
    def __init__(self, start=None, end=None):
        self.products =  pickle.load(open('../../data/products.pkl', 'rb'))
        self._quotes = products.historical_ohlcs()[start:end]
        self.dates = self._quotes.index
        self.symbols = set(self._quotes.columns.get_level_values(0))
        self.invoices = defaultdict(list)
        self.systems = [] #시스템 목록
        
    @property
    def quotes(self):
        return self._quotes
    
    def add_system(self, trader):
        self.systems.append(trader)
        
    def add_invoice(self, invoice):
        self.invoices[invoice.name].append(invoice)
        
    def get_price(self, product,position, quote):
        """
        매수/매도 가격 결정
        전일 장 종료이후 매수/매도 주문을 받음
        매수시: 당일 시가부터 (고가 - 시가)/2 사이의 랜덤 가격
        매도시: 당일 시가부터 (시가-저가)/2 사이의 랜덤 가격
        
        """
        bound = quote['high'] if position == BUY else quote['low']
        limit = (bound+quote['open'])/2
        price = int(np.random.uniform(quote['open'], limit)/product.tickunit) * product.tickunit
        return price
    
    
    def buy(self, name, symbol, position, quote):
        """ 매수 주문 처리
        입력받은 quote로부터 매수가격 결정,
        명세서 생성 및 반환
        """
        
        price = self.get_price(quote, product)
        date = quote.name
        pk = len(invoices)
        invoice = Invoice(pk, name, product, position, price, date)
        self.add_invoice(invoice)
        return invoice
        
    def sell(self, invoice, quote):
        price = self.get_price(invoice.product,invoice.position, quote)
        date = quote.name
        invoice.exit(price, date)
    
    def run(self):
        for date, quote in self.quotes.iterrows():
            for system in self.systems:
                system.trade(quote)
                
            # 하루 끝나고 리포트 작성 #

In [8]:
class Trader:
    def __init__(self, name, principal):
        self.name = name
        self.asset = principal
        self.systems = []
        
    def add_system(self, system):
        self.systems.append(system)
        
    def account(self):
        pass
    
    def portfolio(self):
        pass

In [9]:
class System:
    def __init__(self, name, principal, exch):
        self.name = name
        self.asset = principal
        self.exch = exch
        self.products = exch.products
        self.signals = dict()
        self.invoices = defaultdict(list)
    
    def buy_order(self, product, price, position, quantity, date):
        for idx in range(quantity):
            invoice = Invoice(product)
        
    
    def sell_order(self):
        pass
    
    def stop_order(self):
        pass
    
    def risk_management(self):
        pass
    
    def trade(self, quote):
        symbol = quote.index.get_level_values(level=0).unique()[0]
        product = self.products[symbol]
        decision = self.strategy(product, quote)
        
    def set_signals(self):
        for quote in quotes
    
    def strategy(self, quote):
        

In [256]:
exch = Exchange(start='20120101',end='20120301')
trader = Trader('yosep', 10000)
system1 = System('long only', 5000, exch)

trader.add_system(system1)
#exch.add_system(system1)

exch.run()

In [205]:
b['ZW']

open     427.50
high     428.25
low      422.50
close    424.75
Name: 2017-10-30 00:00:00, dtype: float64

In [239]:
exch.get_price(products['ZW'],SELL, b['ZW'])

425.5

In [165]:
products['ZW'].tickunit

0.25

In [17]:
d = ohlcs['AD']['close'].dropna().rolling(5).mean()

In [19]:
d

1987-01-13        NaN
1987-01-14        NaN
1987-01-15        NaN
1987-01-16        NaN
1987-01-19    0.28688
1987-01-20    0.28758
1987-01-21    0.28954
1987-01-22    0.29058
1987-01-23    0.29154
1987-01-26    0.29248
1987-01-27    0.29266
1987-01-28    0.29264
1987-01-29    0.29284
1987-01-30    0.29288
1987-02-02    0.29278
1987-02-03    0.29328
1987-02-04    0.29420
1987-02-05    0.29498
1987-02-06    0.29588
1987-02-09    0.29616
1987-02-10    0.29646
1987-02-11    0.29646
1987-02-12    0.29642
1987-02-13    0.29606
1987-02-17    0.29614
1987-02-18    0.29620
1987-02-19    0.29616
1987-02-20    0.29636
1987-02-23    0.29680
1987-02-24    0.29756
               ...   
2017-08-30    0.79232
2017-08-31    0.79318
2017-09-01    0.79448
2017-09-05    0.79570
2017-09-11    0.79688
2017-09-12    0.79924
2017-09-13    0.79976
2017-09-14    0.80012
2017-09-15    0.80020
2017-09-18    0.79940
2017-10-03    0.79554
2017-10-04    0.79314
2017-10-05    0.78908
2017-10-06    0.78442
2017-10-09

In [107]:
for d, q in ohlcs.iloc[-1:].iterrows():
    for a, b in q.groupby(level=0):
        print(b)

AD  open     0.7669
    high     0.7687
    low      0.7651
    close    0.7679
Name: 2017-10-30 00:00:00, dtype: float64
BP  open     1.3150
    high     1.3234
    low      1.3140
    close    1.3219
Name: 2017-10-30 00:00:00, dtype: float64
BR  open     0.30850
    high     0.30875
    low      0.30400
    close    0.30540
Name: 2017-10-30 00:00:00, dtype: float64
CD  open     0.7802
    high     0.7808
    low      0.7778
    close    0.7794
Name: 2017-10-30 00:00:00, dtype: float64
CL  open    NaN
    high    NaN
    low     NaN
    close   NaN
Name: 2017-10-30 00:00:00, dtype: float64
ED  open     98.490
    high     98.495
    low      98.485
    close    98.490
Name: 2017-10-30 00:00:00, dtype: float64
EMD  open     1837.0
     high     1838.8
     low      1820.2
     close    1824.8
Name: 2017-10-30 00:00:00, dtype: float64
ES  open     2575.50
    high     2578.25
    low      2565.50
    close    2568.25
Name: 2017-10-30 00:00:00, dtype: float64
FBTP  open    NaN
      high

In [115]:
for a,b  in q.groupby(level=0):
    print(b[a])

open     0.7669
high     0.7687
low      0.7651
close    0.7679
Name: 2017-10-30 00:00:00, dtype: float64
open     1.3150
high     1.3234
low      1.3140
close    1.3219
Name: 2017-10-30 00:00:00, dtype: float64
open     0.30850
high     0.30875
low      0.30400
close    0.30540
Name: 2017-10-30 00:00:00, dtype: float64
open     0.7802
high     0.7808
low      0.7778
close    0.7794
Name: 2017-10-30 00:00:00, dtype: float64
open    NaN
high    NaN
low     NaN
close   NaN
Name: 2017-10-30 00:00:00, dtype: float64
open     98.490
high     98.495
low      98.485
close    98.490
Name: 2017-10-30 00:00:00, dtype: float64
open     1837.0
high     1838.8
low      1820.2
close    1824.8
Name: 2017-10-30 00:00:00, dtype: float64
open     2575.50
high     2578.25
low      2565.50
close    2568.25
Name: 2017-10-30 00:00:00, dtype: float64
open    NaN
high    NaN
low     NaN
close   NaN
Name: 2017-10-30 00:00:00, dtype: float64
open     158.450
high     160.300
low      157.750
close    158.325
Na

In [117]:
q = b[a]

In [118]:
q['high'] - q['open']

open     427.50
high     428.25
low      422.50
close    424.75
Name: 2017-10-30 00:00:00, dtype: float64

In [133]:
np.random.uniform(q['open'], (q['high']-q['open'])/2).round(4)

AttributeError: 'float' object has no attribute 'round'

In [136]:
np.round(np.random.uniform(q['open'], (q['high']-q['open'])/2), 4)

227.9061